# Setup IAM for Kinesis

In [1]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sts = boto3.Session().client(service_name="sts", region_name=region)
iam = boto3.Session().client(service_name="iam", region_name=region)

# Create Kinesis Role

In [2]:
iam_kinesis_role_name = "DSOAWS_Kinesis"

In [3]:
iam_kinesis_role_passed = False

In [4]:
assume_role_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {"Effect": "Allow", "Principal": {"Service": "kinesis.amazonaws.com"}, "Action": "sts:AssumeRole"},
        {"Effect": "Allow", "Principal": {"Service": "firehose.amazonaws.com"}, "Action": "sts:AssumeRole"},
        {"Effect": "Allow", "Principal": {"Service": "kinesisanalytics.amazonaws.com"}, "Action": "sts:AssumeRole"},
    ],
}

In [5]:
import json
import time

from botocore.exceptions import ClientError

try:
    iam_role_kinesis = iam.create_role(
        RoleName=iam_kinesis_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description="DSOAWS Kinesis Role",
    )
    print("Role succesfully created.")
    iam_kinesis_role_passed = True
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        iam_role_kinesis = iam.get_role(RoleName=iam_kinesis_role_name)
        print("Role already exists. That is OK.")
        iam_kinesis_role_passed = True
    else:
        print("Unexpected error: %s" % e)

time.sleep(30)

Role already exists. That is OK.


In [6]:
iam_role_kinesis_name = iam_role_kinesis["Role"]["RoleName"]
print("Role Name: {}".format(iam_role_kinesis_name))

Role Name: DSOAWS_Kinesis


In [7]:
iam_role_kinesis_arn = iam_role_kinesis["Role"]["Arn"]
print("Role ARN: {}".format(iam_role_kinesis_arn))

Role ARN: arn:aws:iam::079002598131:role/DSOAWS_Kinesis


In [8]:
account_id = sts.get_caller_identity()["Account"]

# Specify Stream Name

In [9]:
stream_name = "dsoaws-kinesis-data-stream"

# Specify Firehose Name

In [10]:
firehose_name = "dsoaws-kinesis-data-firehose"

# Specify Lambda Function Name

In [11]:
lambda_fn_name_cloudwatch = "DeliverKinesisAnalyticsToCloudWatch"

In [12]:
lambda_fn_name_invoke_sm_endpoint = "InvokeSageMakerEndpointFromKinesis"

In [13]:
lambda_fn_name_sns = "PushNotificationToSNS"

# Create Policy

In [14]:
kinesis_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:AbortMultipartUpload",
                "s3:GetBucketLocation",
                "s3:GetObject",
                "s3:ListBucket",
                "s3:ListBucketMultipartUploads",
                "s3:PutObject",
            ],
            "Resource": ["arn:aws:s3:::{}".format(bucket), "arn:aws:s3:::{}/*".format(bucket)],
        },
        {
            "Effect": "Allow",
            "Action": ["logs:PutLogEvents"],
            "Resource": ["arn:aws:logs:{}:{}:log-group:/*".format(region, account_id)],
        },
        {
            "Effect": "Allow",
            "Action": [
                "kinesis:*",
            ],
            "Resource": ["arn:aws:kinesis:{}:{}:stream/{}".format(region, account_id, stream_name)],
        },
        {
            "Effect": "Allow",
            "Action": [
                "firehose:*",
            ],
            "Resource": ["arn:aws:firehose:{}:{}:deliverystream/{}".format(region, account_id, firehose_name)],
        },
        {
            "Effect": "Allow",
            "Action": [
                "kinesisanalytics:*",
            ],
            "Resource": ["*"],
        },
        {
            "Sid": "UseLambdaFunction",
            "Effect": "Allow",
            "Action": ["lambda:InvokeFunction", "lambda:GetFunctionConfiguration"],
            "Resource": ["*"],
        },
        {"Effect": "Allow", "Action": "iam:PassRole", "Resource": ["arn:aws:iam::*:role/service-role/kinesis*"]},
    ],
}

print(json.dumps(kinesis_policy_doc, indent=4, sort_keys=True, default=str))

{
    "Statement": [
        {
            "Action": [
                "s3:AbortMultipartUpload",
                "s3:GetBucketLocation",
                "s3:GetObject",
                "s3:ListBucket",
                "s3:ListBucketMultipartUploads",
                "s3:PutObject"
            ],
            "Effect": "Allow",
            "Resource": [
                "arn:aws:s3:::sagemaker-us-east-1-079002598131",
                "arn:aws:s3:::sagemaker-us-east-1-079002598131/*"
            ]
        },
        {
            "Action": [
                "logs:PutLogEvents"
            ],
            "Effect": "Allow",
            "Resource": [
                "arn:aws:logs:us-east-1:079002598131:log-group:/*"
            ]
        },
        {
            "Action": [
                "kinesis:*"
            ],
            "Effect": "Allow",
            "Resource": [
                "arn:aws:kinesis:us-east-1:079002598131:stream/dsoaws-kinesis-data-stream"
            ]
        },
     

# Update Policy

In [15]:
import time

response = iam.put_role_policy(
    RoleName=iam_role_kinesis_name, PolicyName="DSOAWS_KinesisPolicy", PolicyDocument=json.dumps(kinesis_policy_doc)
)

time.sleep(30)

In [16]:
print(json.dumps(response, indent=4, sort_keys=True, default=str))

{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "206",
            "content-type": "text/xml",
            "date": "Sat, 04 Dec 2021 03:23:35 GMT",
            "x-amzn-requestid": "2c58ed2a-4561-4c95-a873-8617f720ebf1"
        },
        "HTTPStatusCode": 200,
        "RequestId": "2c58ed2a-4561-4c95-a873-8617f720ebf1",
        "RetryAttempts": 0
    }
}


# Create AWS Lambda IAM Role

In [17]:
iam_lambda_role_name = "DSOAWS_Lambda"

In [18]:
iam_lambda_role_passed = False

In [19]:
assume_role_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {"Effect": "Allow", "Principal": {"Service": "lambda.amazonaws.com"}, "Action": "sts:AssumeRole"},
        {"Effect": "Allow", "Principal": {"Service": "kinesisanalytics.amazonaws.com"}, "Action": "sts:AssumeRole"},
    ],
}

In [20]:
import time

from botocore.exceptions import ClientError

try:
    iam_role_lambda = iam.create_role(
        RoleName=iam_lambda_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description="DSOAWS Lambda Role",
    )
    print("Role succesfully created.")
    iam_lambda_role_passed = True
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        iam_role_lambda = iam.get_role(RoleName=iam_lambda_role_name)
        print("Role already exists. This is OK.")
        iam_lambda_role_passed = True
    else:
        print("Unexpected error: %s" % e)

time.sleep(30)

Role already exists. This is OK.


In [21]:
iam_role_lambda_name = iam_role_lambda["Role"]["RoleName"]
print("Role Name: {}".format(iam_role_lambda_name))

Role Name: DSOAWS_Lambda


In [22]:
iam_role_lambda_arn = iam_role_lambda["Role"]["Arn"]
print("Role ARN: {}".format(iam_role_lambda_arn))

Role ARN: arn:aws:iam::079002598131:role/DSOAWS_Lambda


# Create AWS Lambda IAM Policy

In [23]:
lambda_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "UseLambdaFunction",
            "Effect": "Allow",
            "Action": ["lambda:InvokeFunction", "lambda:GetFunctionConfiguration"],
            "Resource": "arn:aws:lambda:{}:{}:function:*".format(region, account_id),
        },
        {"Effect": "Allow", "Action": "cloudwatch:*", "Resource": "*"},
        {"Effect": "Allow", "Action": "sns:*", "Resource": "*"},
        {
            "Effect": "Allow",
            "Action": "logs:CreateLogGroup",
            "Resource": "arn:aws:logs:{}:{}:*".format(region, account_id),
        },
        {"Effect": "Allow", "Action": "sagemaker:InvokeEndpoint", "Resource": "*"},
        {
            "Effect": "Allow",
            "Action": ["logs:CreateLogStream", "logs:PutLogEvents"],
            "Resource": "arn:aws:logs:{}:{}:log-group:/aws/lambda/*".format(region, account_id),
        },
    ],
}

In [24]:
print(json.dumps(lambda_policy_doc, indent=4, sort_keys=True, default=str))

{
    "Statement": [
        {
            "Action": [
                "lambda:InvokeFunction",
                "lambda:GetFunctionConfiguration"
            ],
            "Effect": "Allow",
            "Resource": "arn:aws:lambda:us-east-1:079002598131:function:*",
            "Sid": "UseLambdaFunction"
        },
        {
            "Action": "cloudwatch:*",
            "Effect": "Allow",
            "Resource": "*"
        },
        {
            "Action": "sns:*",
            "Effect": "Allow",
            "Resource": "*"
        },
        {
            "Action": "logs:CreateLogGroup",
            "Effect": "Allow",
            "Resource": "arn:aws:logs:us-east-1:079002598131:*"
        },
        {
            "Action": "sagemaker:InvokeEndpoint",
            "Effect": "Allow",
            "Resource": "*"
        },
        {
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Effect": "Allow",
      

In [25]:
import time

response = iam.put_role_policy(
    RoleName=iam_role_lambda_name, PolicyName="DSOAWS_LambdaPolicy", PolicyDocument=json.dumps(lambda_policy_doc)
)

time.sleep(30)

In [26]:
print(json.dumps(response, indent=4, sort_keys=True, default=str))

{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "206",
            "content-type": "text/xml",
            "date": "Sat, 04 Dec 2021 03:24:35 GMT",
            "x-amzn-requestid": "989bcc40-700c-4db7-bc8d-ab6379e0470b"
        },
        "HTTPStatusCode": 200,
        "RequestId": "989bcc40-700c-4db7-bc8d-ab6379e0470b",
        "RetryAttempts": 0
    }
}


# Store Variables for Next Notebooks

In [27]:
%store stream_name

Stored 'stream_name' (str)


In [28]:
%store firehose_name

Stored 'firehose_name' (str)


In [29]:
%store iam_kinesis_role_name

Stored 'iam_kinesis_role_name' (str)


In [30]:
%store iam_role_kinesis_arn

Stored 'iam_role_kinesis_arn' (str)


In [31]:
%store iam_lambda_role_name

Stored 'iam_lambda_role_name' (str)


In [32]:
%store iam_role_lambda_arn

Stored 'iam_role_lambda_arn' (str)


In [33]:
%store lambda_fn_name_cloudwatch

Stored 'lambda_fn_name_cloudwatch' (str)


In [34]:
%store lambda_fn_name_invoke_sm_endpoint

Stored 'lambda_fn_name_invoke_sm_endpoint' (str)


In [35]:
%store lambda_fn_name_sns

Stored 'lambda_fn_name_sns' (str)


In [36]:
%store iam_kinesis_role_passed

Stored 'iam_kinesis_role_passed' (bool)


In [37]:
%store iam_lambda_role_passed

Stored 'iam_lambda_role_passed' (bool)


In [38]:
%store

Stored variables and their in-db values:
firehose_name                                         -> 'dsoaws-kinesis-data-firehose'
iam_kinesis_role_name                                 -> 'DSOAWS_Kinesis'
iam_kinesis_role_passed                               -> True
iam_lambda_role_name                                  -> 'DSOAWS_Lambda'
iam_lambda_role_passed                                -> True
iam_role_kinesis_arn                                  -> 'arn:aws:iam::079002598131:role/DSOAWS_Kinesis'
iam_role_lambda_arn                                   -> 'arn:aws:iam::079002598131:role/DSOAWS_Lambda'
ingest_create_athena_db_passed                        -> True
ingest_create_athena_table_parquet_passed             -> True
ingest_create_athena_table_tsv_passed                 -> True
lambda_fn_name_cloudwatch                             -> 'DeliverKinesisAnalyticsToCloudWatch'
lambda_fn_name_invoke_sm_endpoint                     -> 'InvokeSageMakerEndpointFromKinesis'
lambda_fn_name_s

# Release Resources

In [39]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [40]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>